PySpark는 Spark를 위한 Python API이다.

Public classes 다음이 있다.

1. SparkContext - Spark 기능에 접근하기 위한 메인 입구
2. RDD - Resilient Distributed Dataset, Spark의 기본 데이터
3. Broadcast - 여러 작업에서 재사용되는 브로드캐스트 변수
4. Accumulator - 값을 추가만 할 수 있는 sharable values
5. SparkConf - 스파크 설정 클래스 
6. SparkFiles - job과 함께 전달되는 파일 접근 클래스
7. StorageLevel - 더 세분화된 캐시 지속 레벨
8. TaskContext - 현재 실행되고 있는 task에 대한 정보 
9. RDDBarrier - barrier execution mode를 실행할 때 RDD를 wrap하는 클래스
10. BarrierTaskContext - barrier execution을 위한 extra 정보와 툴을 제공하는 TaskContext
11. BarrierTaskInfo - Information about a barrioer task

In [5]:
import findspark
findspark.init()

In [7]:
# 스파크 애플리케이션을 위한 설정 다양한 스파크 파라미터를 키 밸류 쌍으로 설정하는데 사용
# SparkConf()로 SparkConf를 생성할 수 있으며, spark Java system에서 value를 읽는다. 
# 어떠한 파라미터든 SparkConf object로 직접 설정할 수 있다.
# 단위 테스트의 경우 SparkConf(false)를 호출하여 외부 설정을 건너 뛰고 시스템 속성에 상관없이 동일한 구성을 얻을 수 있다.

from pyspark import SparkConf, SparkContext
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')

In [8]:
# 스파크 기능에 접근하기 위한 메인 entry point SparkContext는 스파크 클러스터에 대한 연결을 나타내고, RDD를 생성하고 클러스터에 변수를
# broadcast할 수 있다.

# JVM당 하나의 스파크콘텍스트만 활성화해야 합니다.새 스파크콘텍스트를 만들기 전에 활성 스파크콘텍스트를 중지해야 한다.
# SparkContext 인스턴스는 기본적으로 여러 프로세스에서 공유하도록 지원되지 않으며 PySpark는 다중 프로세스 실행을 보장하지 않는다.  
# 동시 처리 목적으로 스레드를 대신 사용해야한다.

In [11]:
sc = pyspark.SparkContext(conf=conf)

In [20]:
import os
from pyspark import SparkFiles
path = 'test.txt'
with open(path, "w") as testFile:
    _ = testFile.write("100")
sc.addFile(path)

def func(iterator):
    with open(SparkFiles.get("test.txt")) as testFile:
        fileVal = int(testFile.readline())
        return [x * fileVal for x in iterator]
sc.parallelize([1, 2, 3, 4]).mapPartitions(func).collect()

[100, 200, 300, 400]

In [22]:
sc.range(5).collect()

[0, 1, 2, 3, 4]

In [23]:
sc.range(2, 4).collect()

[2, 3]

In [24]:
sc.range(1, 7, 2).collect()

[1, 3, 5]

In [26]:
myRDD = sc.parallelize(range(6), 3)
sc.runJob(myRDD, lambda part: [x * x for x in part], [0, 2], True)

[0, 1, 16, 25]

In [29]:
import threading
from time import sleep
result = "Not Set"
lock = threading.Lock()

def map_func(x):
    sleep(100)
    raise Exception("Task should have been cancelled")
    
def start_job(x):
    global result
    try:
        sc.setJobGroup("job_to_cancel", "some description")
        result = sc.parallelize(range(x)).map(map_func).collect()
    except Exception as e:
        result = "Cancelled"
    lock.release()
def stop_job():
    
    sleep(5)
    sc.cancelJobGroup("job_to_cancel")

In [30]:
suppress = lock.acquire()

In [31]:
suppress = threading.Thread(target=start_job, args=(10,)).start()

C:\data\spark\spark-3.0.0-bin-hadoop2.7\python\pyspark\util.py:141: UserWarning: Currently, 'setJobGroup' (set to local properties) with multiple threads does not properly work. 
Internally threads on PVM and JVM are not synced, and JVM thread can be reused for multiple threads on PVM, which fails to isolate local properties for each thread on PVM. 
To work around this, you can set PYSPARK_PIN_THREAD to true (see SPARK-22340). However, note that it cannot inherit the local properties from the parent thread although it isolates each thread on PVM and JVM with its own local properties. 
To work around this, you should manually copy and set the local properties from the parent thread to the child thread when you create another thread.
  warnings.warn(msg, UserWarning)


In [32]:
suppress = threading.Thread(target=stop_job).start()
suppress = lock.acquire()

In [33]:
print(result)

Cancelled


In [36]:
path = os.path.join("sample-text.txt")
with open(path, "w") as testFile:
   _ = testFile.write("Hello world!")
textFile = sc.textFile(path)
textFile.collect()

['Hello world!', 'World!']

In [38]:
path = os.path.join("union-text.txt")
with open(path, "w") as testFile:
   _ = testFile.write("Hello")
textFile = sc.textFile(path)
textFile.collect()

parallelized = sc.parallelize(["World!"])
sorted(sc.union([textFile, parallelized]).collect())

['Hello', 'World!']

In [41]:
# Small files are preferred, as each file will be loaded fully in memory.

dirPath = os.path.join("files")
os.mkdir(dirPath)

with open(os.path.join(dirPath, "1.txt"), "w") as file1:
    _ = file1.write("1")
with open(os.path.join(dirPath, "2.txt"), "w") as file2:
    _ = file2.write("2")

textFiles = sc.wholeTextFiles(dirPath)
sorted(textFiles.collect())

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'files'

In [45]:
# Spark의 기본 추상화인 RDD(Resilient Distributed Dataset).병렬로 작동할 수 있는 불변의 분할된 요소 집합을 나타냅니다.

In [118]:
# aggregate the element of each partition

seqOp = (lambda x, y: (x[0] + 1, x[0] + y))
combOp = (lambda x, y: (0, x[1] + y[1] + x[0] + y[0]))

sc.parallelize([1, 2, 3, 4, 5]).aggregate((0, 0), seqOp, combOp)


(0, 14)

In [122]:
seqOp = (lambda x, y: (x[0] + y, x[1] + 1))
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1]))
sc.parallelize([1, 2, 3, 4]).aggregate((0, 0), seqOp, combOp)

sc.parallelize([12, 1]).aggregate((0, 0), seqOp, combOp)

(13, 2)

In [123]:
m = sc.parallelize([(1, 2), (3, 4)]).collectAsMap()

In [126]:
print(m)

{1: 2, 3: 4}


In [128]:
sc.parallelize([2, 3, 4, 2, 3, 4]).count()

6

In [146]:
n = sc.parallelize(range(1130)).map(str).countApproxDistinct()
900 < n < 1200

False

In [151]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
print(rdd.countByKey())
print(sc.parallelize([1, 2, 1, 2, 2], 2).countByValue())


defaultdict(<class 'int'>, {'a': 2, 'b': 1})
defaultdict(<class 'int'>, {1: 2, 2: 3})


In [152]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.filter(lambda x: x % 2 == 0).collect()

[2, 4]

In [154]:
rdd = sc.parallelize([2, 3, 4])
sorted(rdd.flatMap(lambda x: range(1, x)).collect())

[1, 1, 1, 2, 2, 3]

In [155]:
sorted(rdd.flatMap(lambda x: [(x, x), (x, x)]).collect())

[(2, 2), (2, 2), (3, 3), (3, 3), (4, 4), (4, 4)]

In [156]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
sorted(rdd.groupByKey().mapValues(len).collect())

[('a', 2), ('b', 1)]

In [161]:
sorted(rdd.groupByKey().mapValues(tuple).collect())
sorted(rdd.groupByKey().mapValues(list).collect())

[('a', [1, 1]), ('b', [1])]

In [164]:
rdd = sc.parallelize(['a', 'b', 'c'])
rdd.map(lambda x: (x, 2)).collect()

[('a', 2), ('b', 2), ('c', 2)]

In [169]:
rdd = sc.parallelize([1.0, 5.0, 43.0, 10.0]).max()
# rdd.collect()
rdd
# rdd.max()
# rdd.min()

43.0

In [170]:
from operator import add
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
sorted(rdd.reduceByKey(add).collect())

[('a', 2), ('b', 1)]